
**Title: Prokaryotic and eukaryotic promoters Recognition and Prediction**

Environment Requirement: Due to the poor performance of t-SNE during visualization on CPU runtime (taking an average of 16 minutes to generate a 3x3 grid view scatter plot when k=4), please switch to GPU runtime before running this notebook. This will significantly improve processing speed and efficiency.



#2. Loading the Dataset

We will employ the Python [requests](https://pypi.org/project/requests/)
library and SeqIO from [Biopython](https://biopython.org/) to parse FASTA files efficiently.

**Bio.SeqIO.parse(fasta_io, "fasta") → a function from the Biopython library that parses biological sequence files. It extracts sequences while ignoring header lines (>identifier).
Handles multi-line sequences automatically, reconstructing them into a single string.*


In [ ]:
import requests
!pip install biopython
from Bio import SeqIO
import io

This function fetches and parses a FASTA file from GitHub.

In [2]:
# Function to load and parse FASTA sequences
def load_fasta(filename):

    url = BASE_URL + filename  # Construct full URL
    response = requests.get(url)
    response.raise_for_status()  # Ensure successful download

    sequences = []
    fasta_io = io.StringIO(response.text)

    for record in SeqIO.parse(fasta_io, "fasta"):
        sequences.append((str(record.seq).upper(), datasets[filename]))  # Assign correct label

    return sequences


This step downloads and parses FASTA files from GitHub, extracts nucleotide sequences, assigns labels (1 for promoters, 0 for non-promoters), and structures the data for two experiments:

-Experiment 1:

positive = ‘Arabidopsis_tata.fa’, negative = ‘Arabidopsis_non_prom_big.fa’.

-Experiment 2: Non-TATA promoters vs. non-promoters

positive = ‘Arabidopsis_non_tata.fa’, negative = ‘Arabidopsis_non_prom_big.fa’.




In [ ]:


# Base URL for the dataset
BASE_URL = "https://raw.githubusercontent.com/solovictor/CNNPromoterData/refs/heads/master/"

# Filenames and corresponding labels
datasets = {
    "Arabidopsis_tata.fa": 1,        # Positive (TATA)
    "Arabidopsis_non_tata.fa": 1,    # Positive (Non-TATA)
    "Arabidopsis_non_prom_big.fa": 0 # Negative
}

# Load datasets
positive_tata = load_fasta("Arabidopsis_tata.fa")
positive_non_tata = load_fasta("Arabidopsis_non_tata.fa")
negative = load_fasta("Arabidopsis_non_prom_big.fa")

# Create experiments
experiment_1 = positive_tata + negative
experiment_2 = positive_non_tata + negative

# Display dataset sizes
print(f"Experiment 1: {len(experiment_1)} sequences")
print(f"Experiment 2: {len(experiment_2)} sequences")

# Example sequence preview
print("Sample sequence:", experiment_1[0])


#3. Data Encoding

We will use [itertools](https://docs.python.org/3/library/itertools.html) library to create all nucleotide combinations and BioPython's Seq object provides biological sequence handling.



In [4]:
import itertools
import numpy as np
from collections import Counter
from Bio.Seq import Seq

This function moves one nucleotide at a time, capturing overlapping k-mers as step size = 1 (by default)


In [5]:
# Function to generate k-mers using BioPython
def generate_kmers(sequence, k):
    return [str(Seq(sequence[i:i+k])) for i in range(len(sequence) - k + 1)]

And generate a normalized k-mer frequency vector for a given sequence.

In [6]:
# Function to compute k-mer frequency vector
def kmer_frequency_vector(sequence, k):

    kmer_list = [''.join(p) for p in itertools.product("ACGT", repeat=k)]  # Generate all possible k-mers
    kmer_counts = Counter(generate_kmers(sequence, k))  # Count occurrences using BioPython sequences

    # Normalize frequencies
    total_kmers = sum(kmer_counts.values())
    kmer_freq_vector = {kmer: kmer_counts.get(kmer, 0) / total_kmers for kmer in kmer_list}

    return np.array(list(kmer_freq_vector.values()))  # Convert to numerical format

Then encodes all sequences in a dataset using k-mer frequency vectors.

In [7]:
# Function to encode datasets
def encode_dataset(dataset, k):

    encoded_data = np.array([kmer_frequency_vector(seq, k) for seq, _ in dataset])
    labels = np.array([label for _, label in dataset])  # Extract labels
    return encoded_data, labels

This Step extracts overlapping k-mers (subsequences) using BioPython. Counts and normalizes k-mer occurrences for valid nucleotides (A, C, G, T). Then generates numerical feature vectors for experiment_1 & experiment_2 datasets.


In [ ]:

# Define k values (4 to 6) for bioinformatics analysis, this is Global parameter
k_values = [4, 5, 6]

# Encode Experiment 1 & 2 datasets for multiple k values
encoded_experiments = {k: {
    "experiment_1": encode_dataset(experiment_1, k),
    "experiment_2": encode_dataset(experiment_2, k)
} for k in k_values}

# Display dataset dimensions
for k, data in encoded_experiments.items():
    print(f"\nEncoding for k={k}:")
    print(f"  Experiment 1: {data['experiment_1'][0].shape}, Labels: {data['experiment_1'][1].shape}")
    print(f"  Experiment 2: {data['experiment_2'][0].shape}, Labels: {data['experiment_2'][1].shape}")


**Consideratios*

e.g. Encoding for k=4:
  Experiment 1: (12956, 256), Labels: (12956,)

Meaning 12956 sample count and 256 Features.

t-SNE is not designed for very large datasets and can become computationally expensive for more than 10,000 samples or dimensions are above 50. Need to apply further dimension reduction like PCA.

#4. Understanding your Data

##4.1 Class Distribution
Assess the distribution of instances within each class, distinguishing between positive and negative examples. Determine whether the datasets for these experiments are balanced or exhibit class imbalance.

The ratios suggested in this Google ML Crash Course are typical.

*   60% (majority class)/40% (minority class) split would not be considered problematic
*   90% (majority class)/10% (minority class) split would be high
*   Between normal and high would be considered mild
*   99%/1% or worse would be extreme





In [ ]:
print("Type of experiment_1: ", type(experiment_1)) #take a peek at experiment_1 data type


import pandas as pd

# Convert datasets to DataFrames
df_exp1 = pd.DataFrame(experiment_1, columns=["Sequence", "Label"])
df_exp2 = pd.DataFrame(experiment_2, columns=["Sequence", "Label"])

# Compute class distribution in percentage
exp1_distribution = df_exp1["Label"].value_counts(normalize=True) * 100
exp2_distribution = df_exp2["Label"].value_counts(normalize=True) * 100

#Class distribution summary
exp1_summary = f"Experiment 1 Class Distribution:\nPositive: {exp1_distribution[1]:.2f}%\nNegative: {exp1_distribution[0]:.2f}%\n"
exp2_summary = f"Experiment 2 Class Distribution:\nPositive: {exp2_distribution[1]:.2f}%\nNegative: {exp2_distribution[0]:.2f}%\n"

# Print formatted results
print(exp1_summary)
print(exp2_summary)

##4.2 Visualization

In this step we will

*   Generate a graph representing the positive examples.
*   Generate a graph representing the negative examples.
*   Generate a composite graph that includes both positive and negative examples

It is advisable to investigate the influence of various parameters. For instance, consider altering the value of
 between 4 and 6. Additionally, parameters such as ‘perplexity’ and ‘early_exaggeration’ are critical and warrant exploration due to their potential impact on the results.



Check  CUDA Version

In [ ]:
import torch
print(torch.cuda.is_available())  # Check if CUDA is enabled
print(torch.version.cuda)  # Display the CUDA version


Necessary pip installation commands for running it on NVIDIA GPUs.


In [ ]:
!pip install cuml-cu12 --extra-index-url=https://pypi.nvidia.com
!pip install cupy-cuda12x
!pip uninstall -y numpy
!pip install --upgrade "numpy<2.0.0"




We will use [cuml](https://github.com/rapidsai/cuml) for GPU-accelerated t-SNE. And [cupy](https://github.com/cupy/cupy) to handle GPU-based arrays (cp.ndarray) instead of CPU-based NumPy arrays (np.ndarray), which converts PCA-reduced NumPy data → CuPy arrays before passing to TSNE.

**If you encouter ImportError: cannot import name 'intp' from 'numpy._core' (/usr/local/lib/python3.11/dist-packages/numpy/_core/__init__.py), Downgrade NumPy to a Compatible Version*


In [16]:
import matplotlib.pyplot as plt
import importlib
from cuml.manifold import TSNE  # GPU-accelerated t-SNE
from sklearn.decomposition import PCA
import cupy as cp



This fucntion projects high-dimensional k-mer encoded data into 2D space using PCA + GPU t-SNE and plots results in a grid.



In [24]:
import os

# Create a directory to save plots
output_dir = "tsne_plots"
os.makedirs(output_dir, exist_ok=True)

In [34]:
# Function to perform GPU-accelerated t-SNE visualization
def plot_tsne_grid(encoded_data, labels, title, perplexities, exaggerations, pca_components=30):

    num_rows = len(perplexities) * len(exaggerations)  # Each perplexity & exaggeration pair gets its own row
    num_cols = 3  # Columns represent Positive, Negative, and Combined
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(6 * num_cols, 6 * num_rows))  # Increase figure size for better visibility

    row_index = 0
    for perplexity in perplexities:
        for early_exaggeration in exaggerations:

            # Apply PCA for dimensionality reduction if needed
            if encoded_data.shape[1] > pca_components:
                pca = PCA(n_components=pca_components, random_state=42)
                reduced_data = pca.fit_transform(encoded_data)
            else:
                reduced_data = encoded_data

            # Convert data to GPU format using CuPy
            reduced_data_gpu = cp.asarray(reduced_data)

            # Perform optimized t-SNE projection using cuML
            tsne = TSNE(n_components=2, perplexity=perplexity, early_exaggeration=early_exaggeration, random_state=42)
            transformed_data = tsne.fit_transform(reduced_data_gpu)
            transformed_data = cp.asnumpy(transformed_data)  # Convert back to NumPy for plotting

            # Split data into positive and negative classes
            pos_data = transformed_data[labels == 1]
            neg_data = transformed_data[labels == 0]

            # Plot Positive Examples
            ax_pos = axes[row_index, 0]
            ax_pos.scatter(pos_data[:, 0], pos_data[:, 1], label="Positive", alpha=0.7, color="navy", s=12)
            ax_pos.set_title(f"Positive - Perp={perplexity}, Exag={early_exaggeration}")


            # Plot Negative Examples
            ax_neg = axes[row_index, 1]
            ax_neg.scatter(neg_data[:, 0], neg_data[:, 1], label="Negative", alpha=0.7, color="pink", s=12)
            ax_neg.set_title(f"Negative - Perp={perplexity}, Exag={early_exaggeration}")


            # Plot Combined Examples
            ax_comb = axes[row_index, 2]
            ax_comb.scatter(neg_data[:, 0], neg_data[:, 1], label="Negative", alpha=0.7, color="pink", s=12)
            ax_comb.scatter(pos_data[:, 0], pos_data[:, 1], label="Positive", alpha=0.7, color="navy", s=12)
            ax_comb.set_title(f"Combined - Perp={perplexity}, Exag={early_exaggeration}")


            row_index += 1

    # Add a single shared legend for all subplots
    handles, labels = ax_comb.get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper right')

    plt.suptitle(title, fontsize=16, y=1.02)
    plt.tight_layout()

    # Save the plot to the output directory
    save_path = os.path.join(output_dir, f"{title.replace(' ', '_')}.png")
    plt.savefig(save_path)
    plt.show()

**The perplexity parameter controls how many neighbors each point considers when computing similarity, which defines the balance between local and global structure in the visualization.*


*   Normally Perplexity should be ≈ dataset size / 3.
*   Low (5-10)	Focuses on local clusters but may miss global structure.
*   Medium (20-50)	Good balance between local and global structure.
*   High (50-100)	Preserves global structure, but local clusters may merge incorrectly.



**The early_exaggeration controls how widely separated clusters are at the beginning of optimization. It helps t-SNE spread out the data before refining the structure.*

*   Default early_exaggeration=12
*   Low (1-10)	Clusters form slowly, potentially overlapping.
*   Medium (12-24)	Good balance, allowing structure to spread initially.
*   High (25-50)	Overseparates clusters, making structure less natural.








In [ ]:
import shutil

# k values falready defined in step3: k_values = [4, 5, 6]
perplexities = [5, 10, 20]  # Adjusted for large datasets
early_exaggerations = [12, 24, 48]  # Different exaggeration values to test

# Perform GPU-accelerated t-SNE visualization for different k values
for k in k_values:
    print(f"\n### t-SNE Grid Visualization for k={k} ###")

    # Load encoded datasets from encoded_experiments
    encoded_exp1, labels_exp1 = encoded_experiments[k]["experiment_1"]
    encoded_exp2, labels_exp2 = encoded_experiments[k]["experiment_2"]

    # Generate grid view for Experiment 1
    plot_tsne_grid(encoded_exp1, labels_exp1, title=f"t-SNE Grid (Experiment 1, k={k})",
                   perplexities=perplexities, exaggerations=early_exaggerations)

    # Generate grid view for Experiment 2
    plot_tsne_grid(encoded_exp2, labels_exp2, title=f"t-SNE Grid (Experiment 2, k={k})",
                   perplexities=perplexities, exaggerations=early_exaggerations)

# Zip the saved plots
zip_path = "tsne_plots.zip"
shutil.make_archive("tsne_plots", 'zip', output_dir)
print(f"All plots have been saved and zipped to {zip_path}")


#5. Data Partitioning

Divide dataset into distinct training and testing subsets, allocating 20% of the data specifically for testing purposes.

In [36]:
from sklearn.model_selection import train_test_split

# Function to partition dataset into training and testing sets
def partition_data(encoded_data, labels, test_size=0.2, random_state=42):

    X_train, X_test, y_train, y_test = train_test_split(encoded_data, labels, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

Create partitioned datasets for each k-mer encoding

In [37]:

partitioned_experiments = {
    k: {
        "experiment_1": partition_data(*encoded_experiments[k]["experiment_1"]),
        "experiment_2": partition_data(*encoded_experiments[k]["experiment_2"]),
    }
    for k in k_values
}


#6. Training and Testing


For each experimental condition:

*   Train a logistic regression.
*   Measure the performance of your model on the test set. Use the method classification_report to show the precision, recall, and f1-score.
*   Show the resulting confusion matrix.








Import library for LogisticRegression and confusion matrix.

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

Trains a logistic regression model and evaluates its performance.

In [39]:
# Function to train and evaluate logistic regression
def train_and_evaluate(X_train, X_test, y_train, y_test, title):

    # Train Logistic Regression model
    clf = LogisticRegression(max_iter=500, random_state=42)
    clf.fit(X_train, y_train)

    # Predict on test set
    y_pred = clf.predict(X_test)

    # Print classification report
    print(f"\n### Classification Report for {title} ###")
    print(classification_report(y_test, y_pred))

    # Compute confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # Plot confusion matrix
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Negative", "Positive"], yticklabels=["Negative", "Positive"])
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title(f"Confusion Matrix for {title}")
    plt.show()

Train and test model for each experimental condition

In [ ]:

for k in k_values:
    print(f"\n### Training Logistic Regression for k={k} ###")

    # Load partitioned datasets
    (X_train_exp1, X_test_exp1, y_train_exp1, y_test_exp1) = partitioned_experiments[k]["experiment_1"]
    (X_train_exp2, X_test_exp2, y_train_exp2, y_test_exp2) = partitioned_experiments[k]["experiment_2"]

    # Train and evaluate for Experiment 1
    train_and_evaluate(X_train_exp1, X_test_exp1, y_train_exp1, y_test_exp1, title=f"Experiment 1 (k={k})")

    # Train and evaluate for Experiment 2
    train_and_evaluate(X_train_exp2, X_test_exp2, y_train_exp2, y_test_exp2, title=f"Experiment 2 (k={k})")
